# Standard Kalman Filter Demo

This notebook demonstrates a basic Kalman filter implementation using pykal.

In [ ]:
from pykal import DynamicalSystem
from pykal.utilities.estimators.kf import KF
import numpy as np
import matplotlib.pyplot as plt

## Summary

This demo shows:
1. **System modeling** using pykal's `DynamicalSystem`
2. **Kalman filtering** with `KF.f()` for state estimation